In [148]:
import tensorflow as tf
import json
import numpy as np
from tensorflow import keras as tfk
import os
from pymongo import MongoClient
import pandas as pd

In [149]:
conversation_prefixes = ['conversation1', 'conversation2', 'conversation3']
conversation_file_contents = []
for y in conversation_prefixes:
    conversation_file_contents.append(pd.read_csv('conversations_kaggle/'+str(y)+'.txt', delimiter='\n\n', header=None\
                                                , engine='python'))

with open('conversations_kaggle/appos.json', 'r') as conjunctions_file:
    conjunctions = json.load(conjunctions_file)
    conjunctions = conjunctions['appos']

In [157]:
for y, x in enumerate(conversation_file_contents):
    x.to_csv('conversations_kaggle/'+conversation_prefixes[y]+'.csv', index=True)

In [158]:
conversation_csv_contents = []
for z in conversation_prefixes:
    conversation_csv_contents.append(pd.read_csv('conversations_kaggle/'+z+'.csv', index_col='Unnamed: 0'))

In [159]:
def make_dict_real(file_contents, multiple_files=True):
    list_of_dialogue = []
    if multiple_files == False:
        j = file_contents['0'].tolist()
        k = []
        for h in j:
            l = dict({h.split(':')[0]: h.split(': ')[-1]})
            k.append(l)
        list_of_dialogue = k
        return list_of_dialogue
    else:
        for i in file_contents:
            j = i['0'].tolist()
            k = []
            for h in j:
                l = dict({h.split(':')[0]: h.split(': ')[-1]})
                k.append(l)
            list_of_dialogue.append(k)
    return list_of_dialogue

In [211]:
def prepare_data_for_tokenizing(words, normalize_dict):
    words_seen_before = ''
    for q in words:
        qrs = ''
        for r in q:
            qrs += list(r.items())[0][1]
        srq = qrs.lower().split(' ')
        for s in normalize_dict:
            signal = 'Y'
            while signal == 'Y':
                try:
                    qrs[srq.index(s)] = normalize_dict[s]
                except:
                    'KeyError'
                    signal ='N'
        words_seen_before += ' '+qrs
    return words_seen_before.split(' ')[1:]

In [212]:
bots_ownDict = tf.keras.preprocessing.text.Tokenizer(\
                                                     num_words=9001\
                                                     , filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'\
                                                     , lower=False\
                                                     ,char_level=False, oov_token='<<UNKNOWN>>', document_count=1\
                                                    )

In [214]:
words_dict_within_list = make_dict_real(conversation_csv_contents)
list_of_cleaned_words = prepare_data_for_tokenizing(words_dict_within_list, conjunctions)

In [216]:
bots_ownDict.fit_on_texts(list_of_cleaned_words)

In [219]:
list(bots_ownDict.word_index.items())[0:20]

[('<<UNKNOWN>>', 1),
 ('to', 2),
 ('the', 3),
 ('I', 4),
 ('of', 5),
 ('our', 6),
 ('is', 7),
 ('we', 8),
 ('in', 9),
 ('a', 10),
 ('and', 11),
 ('you', 12),
 ('on', 13),
 ('that', 14),
 ('have', 15),
 ('it', 16),
 ('with', 17),
 ('sales', 18),
 ('customers', 19),
 ('It', 20)]

In [224]:
with open('tokenizerWordIndex_dialogue.json', 'a') as tokenized_file:
    json.dump(bots_ownDict.to_json(), tokenized_file)